In [ ]:
#imports 
import numpy as np
import pandas as pd
from scipy import stats

In [ ]:
data = pd.read_csv("../input/spotify-dataset-19212020-160k-tracks/data_o.csv") 

In [ ]:
data.head()
data.info()

In [ ]:
#removing dupliates
data['artists+name'] = data.apply(lambda row: row['artists'] + row['name'], axis=1)
df = data[data['artists+name'].duplicated()]

In [ ]:
indices = []

for name in df['artists+name'].unique():
    subset = data[data['artists+name'] == name].copy()
    m = subset['popularity'].max()
    index = subset[subset['popularity'] == m].index[0]
    indices.append(index)

In [ ]:
data_bis = data.loc[indices].copy()

In [ ]:
data_ter = data.copy()
for i, row in data.iterrows():
    if row['artists+name'] in df['artists+name'].unique():
        data_ter.drop(index=i, inplace=True)
        
frames = [data_bis, data_ter]
data_four = pd.concat(frames)

In [ ]:
#remove unnecessary columns
data_four.drop(columns=['release_date','id','artists+name', 'loudness'], inplace=True, axis=1)

In [ ]:
def remove_outlier(data, threshold):

    data = data.copy()

    thresholdz = stats.norm.ppf(threshold)
    data = data[(np.abs(stats.zscore(data['duration_ms'])) < thresholdz)]
  
    return data


In [ ]:
#remove outliers from data
final = remove_outlier(data_four, .9999)

In [ ]:
#map artists to the average popularity of their song
artists_and_pop = {}
count = 0
train_mean_pop = final['popularity'].mean()

for artist in final['artists'].unique():
    temp = final[final['artists'] == artist]['popularity'].copy()
    if len(temp) > 1:
        artists_and_pop[artist] = temp.mean()
    elif len(temp) == 1:
        artists_and_pop[artist] = train_mean_pop
    else:
        print('Stopping iteration due to unexpected result.')
        break

In [ ]:
final['artists'] = final['artists'].map(artists_and_pop)

In [ ]:
#save 
final.to_csv('data/songs.csv', index=False)